In [8]:
from theano.sandbox import cuda

In [9]:
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [13]:
path = 'data/dogscats/'
model_path = path + 'model/'

In [15]:
model = vgg_ft(2)

In [16]:
model.load_weights(model_path+'fintune.h5')

IOError: Unable to open file (Unable to open file: name = 'data/dogscats/model/fintune.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)